In [1]:
import numpy as np
import pandas as pd
import importlib
import sys

from pytorch_tabnet.tab_model import TabNetClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from data_processing import timer, one_hot_encoder, application_train_test, bureau_and_balance, previous_applications, pos_cash, installments_payments, credit_card_balance

from collections import defaultdict, Counter

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import wget
from pathlib import Path

import gc
import time
from contextlib import contextmanager
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

importlib.reload(sys.modules[Classifier_MLP.__module__])
importlib.reload(sys.modules[Classifier_FCN.__module__])

from matplotlib import pyplot as plt
%matplotlib inline

In [7]:
LIGHTGBM_PARAMS = {
    'boosting_type': 'goss',
    'n_estimators': 2000,
    'learning_rate': 0.005134,
    'num_leaves': 54,
    'max_depth': 10,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 1,
    'is_unbalance': False,
    'silent':-1,
    'verbose': 1,
    'nthread': 4,
    'random_state': 1234,
    'metric' : 'auc',
    'n_jobs': 4
}

In [8]:
XGBOOST_PARAMS = {
    "learning_rate" : 0.01, 
    "n_estimators" : 2000, 
    "max_depth" : 4, 
    "min_child_weight" : 5, 
    "subsample" : 0.8, 
    "colsample_bytree" : 0.8, 
    "objective" : 'binary:logistic', 
    "scale_pos_weight" : 2.5,
    "reg_lambda" : 1.2,
#     "nthread" : 4,
    "seed" : 1234,
    'metric' : 'auc',
}

In [ ]:
def eval_model(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(0)
    X = df.drop(['TARGET','SK_ID_CURR'], axis=1).values
    y = df["TARGET"].values
    
    train_ratio = 0.8
    validation_ratio = 0.1
    test_ratio = 0.1

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))
    
    clf_lgbm = LGBMClassifier(**LIGHTGBM_PARAMS)
    clf_lgbm.fit(X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        eval_metric= 'auc', 
        verbose=1, 
        early_stopping_rounds=200)
    
    y_pred_lgmb = clf_lgbm.predict_proba(X_test)[:, 1]
    print("LightGBM AUC score is:", roc_auc_score(y_test, y_pred_lgmb))
    
    clf_xgb = XGBClassifier(**XGBOOST_PARAMS)
    clf_xgb.fit(X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        eval_metric= 'auc', 
        verbose=1, 
        early_stopping_rounds=200)
    y_pred_xgb = clf_xgb.predict_proba(X_test)[:, 1]
    print("XGBoost AUC score is:", roc_auc_score(y_test, y_pred_xgb))
    
    clf_tabnet = TabNetClassifier(optimizer_fn=torch.optim.Adam)
    max_epochs = 25 if not os.getenv("CI", False) else 2
    clf_tabnet.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False
    ) 
    y_pred_tabnet = clf_tabnet.predict_proba(X_test)[:, 1]
    print("TabNet AUC score is:", roc_auc_score(y_test, y_pred_tabnet))

# Credit card balance

In [2]:
num_rows = None
df = application_train_test(num_rows)

with timer("Process credit card balance"):
    cc = credit_card_balance(num_rows)
    print("Credit card balance df shape:", cc.shape)
    df = df.join(cc, how='left', on='SK_ID_CURR')
    del cc
    gc.collect()

Train samples: 307511
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 20s


In [3]:
df= df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

In [4]:
X = df.drop(['TARGET','SK_ID_CURR'], axis=1).values
y = df["TARGET"].values

In [5]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [9]:
clf_lgbm = LGBMClassifier(**LIGHTGBM_PARAMS)

In [10]:
clf_lgbm.fit(X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        eval_metric= 'auc', 
        verbose=1, 
        early_stopping_rounds=200)

[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=4. Current value: num_threads=4
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=4. Current value: num_threads=4
[LightGBM] [Info] Number of positive: 19806, number of negative: 226202
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.195741 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 34087
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 346
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=4. Current value: num_threads=4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080510 -> initscore=-2.435443
[LightGBM] [Info] Start training from score -2.435443
[1]	training's auc: 0.662688	valid_1's auc: 0.649617
Tr

[136]	training's auc: 0.760433	valid_1's auc: 0.750783
[137]	training's auc: 0.760484	valid_1's auc: 0.750832
[138]	training's auc: 0.760604	valid_1's auc: 0.750913
[139]	training's auc: 0.760654	valid_1's auc: 0.750937
[140]	training's auc: 0.760752	valid_1's auc: 0.75094
[141]	training's auc: 0.76073	valid_1's auc: 0.750893
[142]	training's auc: 0.76085	valid_1's auc: 0.750986
[143]	training's auc: 0.760862	valid_1's auc: 0.750988
[144]	training's auc: 0.760839	valid_1's auc: 0.750945
[145]	training's auc: 0.760835	valid_1's auc: 0.750941
[146]	training's auc: 0.760899	valid_1's auc: 0.751047
[147]	training's auc: 0.760906	valid_1's auc: 0.751043
[148]	training's auc: 0.760875	valid_1's auc: 0.751007
[149]	training's auc: 0.760855	valid_1's auc: 0.750939
[150]	training's auc: 0.760989	valid_1's auc: 0.750984
[151]	training's auc: 0.760995	valid_1's auc: 0.750996
[152]	training's auc: 0.761039	valid_1's auc: 0.750993
[153]	training's auc: 0.761168	valid_1's auc: 0.751087
[154]	trainin

[287]	training's auc: 0.76831	valid_1's auc: 0.754197
[288]	training's auc: 0.768377	valid_1's auc: 0.754225
[289]	training's auc: 0.768424	valid_1's auc: 0.75424
[290]	training's auc: 0.768511	valid_1's auc: 0.754293
[291]	training's auc: 0.768605	valid_1's auc: 0.754301
[292]	training's auc: 0.768694	valid_1's auc: 0.754345
[293]	training's auc: 0.768744	valid_1's auc: 0.754378
[294]	training's auc: 0.768768	valid_1's auc: 0.754394
[295]	training's auc: 0.768902	valid_1's auc: 0.754515
[296]	training's auc: 0.768942	valid_1's auc: 0.75449
[297]	training's auc: 0.769046	valid_1's auc: 0.754515
[298]	training's auc: 0.769075	valid_1's auc: 0.754553
[299]	training's auc: 0.769101	valid_1's auc: 0.754566
[300]	training's auc: 0.769197	valid_1's auc: 0.754621
[301]	training's auc: 0.769246	valid_1's auc: 0.754645
[302]	training's auc: 0.769275	valid_1's auc: 0.754646
[303]	training's auc: 0.76932	valid_1's auc: 0.754667
[304]	training's auc: 0.76934	valid_1's auc: 0.754664
[305]	training'

[437]	training's auc: 0.775614	valid_1's auc: 0.756884
[438]	training's auc: 0.775666	valid_1's auc: 0.756922
[439]	training's auc: 0.775732	valid_1's auc: 0.756959
[440]	training's auc: 0.775768	valid_1's auc: 0.756962
[441]	training's auc: 0.775803	valid_1's auc: 0.756957
[442]	training's auc: 0.775864	valid_1's auc: 0.756965
[443]	training's auc: 0.775903	valid_1's auc: 0.756979
[444]	training's auc: 0.775949	valid_1's auc: 0.756971
[445]	training's auc: 0.775993	valid_1's auc: 0.756984
[446]	training's auc: 0.776013	valid_1's auc: 0.756995
[447]	training's auc: 0.776055	valid_1's auc: 0.757014
[448]	training's auc: 0.776107	valid_1's auc: 0.757041
[449]	training's auc: 0.776148	valid_1's auc: 0.757052
[450]	training's auc: 0.776193	valid_1's auc: 0.757075
[451]	training's auc: 0.776278	valid_1's auc: 0.757123
[452]	training's auc: 0.776319	valid_1's auc: 0.757118
[453]	training's auc: 0.776381	valid_1's auc: 0.75715
[454]	training's auc: 0.776437	valid_1's auc: 0.757162
[455]	train

[587]	training's auc: 0.781829	valid_1's auc: 0.758984
[588]	training's auc: 0.781857	valid_1's auc: 0.758989
[589]	training's auc: 0.78189	valid_1's auc: 0.75899
[590]	training's auc: 0.78193	valid_1's auc: 0.759001
[591]	training's auc: 0.781974	valid_1's auc: 0.759004
[592]	training's auc: 0.782026	valid_1's auc: 0.759015
[593]	training's auc: 0.782065	valid_1's auc: 0.759024
[594]	training's auc: 0.782085	valid_1's auc: 0.75903
[595]	training's auc: 0.78213	valid_1's auc: 0.759039
[596]	training's auc: 0.782164	valid_1's auc: 0.759034
[597]	training's auc: 0.782231	valid_1's auc: 0.759056
[598]	training's auc: 0.782264	valid_1's auc: 0.759057
[599]	training's auc: 0.782313	valid_1's auc: 0.759056
[600]	training's auc: 0.782369	valid_1's auc: 0.759097
[601]	training's auc: 0.782417	valid_1's auc: 0.759106
[602]	training's auc: 0.782456	valid_1's auc: 0.759114
[603]	training's auc: 0.782499	valid_1's auc: 0.759137
[604]	training's auc: 0.782539	valid_1's auc: 0.759161
[605]	training'

...

[1479]	training's auc: 0.814255	valid_1's auc: 0.76611
[1480]	training's auc: 0.814284	valid_1's auc: 0.766122
[1481]	training's auc: 0.814322	valid_1's auc: 0.766137
[1482]	training's auc: 0.81436	valid_1's auc: 0.766146
[1483]	training's auc: 0.81439	valid_1's auc: 0.766148
[1484]	training's auc: 0.814425	valid_1's auc: 0.766154
[1485]	training's auc: 0.814456	valid_1's auc: 0.76616
[1486]	training's auc: 0.814488	valid_1's auc: 0.76616
[1487]	training's auc: 0.814518	valid_1's auc: 0.766152
[1488]	training's auc: 0.814542	valid_1's auc: 0.766155
[1489]	training's auc: 0.814568	valid_1's auc: 0.766164
[1490]	training's auc: 0.814606	valid_1's auc: 0.766157
[1491]	training's auc: 0.814642	valid_1's auc: 0.766147
[1492]	training's auc: 0.814667	valid_1's auc: 0.766153
[1493]	training's auc: 0.814699	valid_1's auc: 0.766161
[1494]	training's auc: 0.814717	valid_1's auc: 0.766167
[1495]	training's auc: 0.814745	valid_1's auc: 0.766168
[1496]	training's auc: 0.814772	valid_1's auc: 0.7661

[1627]	training's auc: 0.818614	valid_1's auc: 0.766732
[1628]	training's auc: 0.818647	valid_1's auc: 0.766739
[1629]	training's auc: 0.818679	valid_1's auc: 0.766735
[1630]	training's auc: 0.818705	valid_1's auc: 0.766757
[1631]	training's auc: 0.818729	valid_1's auc: 0.766774
[1632]	training's auc: 0.818751	valid_1's auc: 0.766772
[1633]	training's auc: 0.818781	valid_1's auc: 0.766771
[1634]	training's auc: 0.818806	valid_1's auc: 0.766763
[1635]	training's auc: 0.81883	valid_1's auc: 0.766766
[1636]	training's auc: 0.81886	valid_1's auc: 0.766769
[1637]	training's auc: 0.818892	valid_1's auc: 0.766772
[1638]	training's auc: 0.818918	valid_1's auc: 0.766778
[1639]	training's auc: 0.81894	valid_1's auc: 0.766785
[1640]	training's auc: 0.818969	valid_1's auc: 0.766785
[1641]	training's auc: 0.818999	valid_1's auc: 0.766785
[1642]	training's auc: 0.819028	valid_1's auc: 0.766785
[1643]	training's auc: 0.81906	valid_1's auc: 0.766795
[1644]	training's auc: 0.81909	valid_1's auc: 0.7667

[1775]	training's auc: 0.822875	valid_1's auc: 0.767153
[1776]	training's auc: 0.822902	valid_1's auc: 0.767152
[1777]	training's auc: 0.822934	valid_1's auc: 0.767165
[1778]	training's auc: 0.822959	valid_1's auc: 0.767166
[1779]	training's auc: 0.822982	valid_1's auc: 0.767178
[1780]	training's auc: 0.823006	valid_1's auc: 0.767191
[1781]	training's auc: 0.823048	valid_1's auc: 0.767205
[1782]	training's auc: 0.823073	valid_1's auc: 0.767207
[1783]	training's auc: 0.823095	valid_1's auc: 0.767208
[1784]	training's auc: 0.823124	valid_1's auc: 0.767201
[1785]	training's auc: 0.823156	valid_1's auc: 0.767215
[1786]	training's auc: 0.823167	valid_1's auc: 0.767218
[1787]	training's auc: 0.823195	valid_1's auc: 0.767218
[1788]	training's auc: 0.823224	valid_1's auc: 0.767215
[1789]	training's auc: 0.823248	valid_1's auc: 0.76721
[1790]	training's auc: 0.823278	valid_1's auc: 0.767223
[1791]	training's auc: 0.823311	valid_1's auc: 0.767216
[1792]	training's auc: 0.823335	valid_1's auc: 0.

[1923]	training's auc: 0.826751	valid_1's auc: 0.76754
[1924]	training's auc: 0.826778	valid_1's auc: 0.767547
[1925]	training's auc: 0.826805	valid_1's auc: 0.76755
[1926]	training's auc: 0.826828	valid_1's auc: 0.767545
[1927]	training's auc: 0.82685	valid_1's auc: 0.767544
[1928]	training's auc: 0.826872	valid_1's auc: 0.767543
[1929]	training's auc: 0.826895	valid_1's auc: 0.76754
[1930]	training's auc: 0.826918	valid_1's auc: 0.767545
[1931]	training's auc: 0.826944	valid_1's auc: 0.76755
[1932]	training's auc: 0.826969	valid_1's auc: 0.767551
[1933]	training's auc: 0.827	valid_1's auc: 0.767547
[1934]	training's auc: 0.827026	valid_1's auc: 0.767552
[1935]	training's auc: 0.827047	valid_1's auc: 0.767559
[1936]	training's auc: 0.827086	valid_1's auc: 0.767574
[1937]	training's auc: 0.82712	valid_1's auc: 0.767579
[1938]	training's auc: 0.827145	valid_1's auc: 0.767584
[1939]	training's auc: 0.827171	valid_1's auc: 0.76759
[1940]	training's auc: 0.827193	valid_1's auc: 0.767598
[1

LGBMClassifier(boosting_type='goss', colsample_bytree=0.508716,
               is_unbalance=False, learning_rate=0.005134, max_depth=10,
               metric='auc', min_split_gain=0.024766, n_estimators=2000,
               n_jobs=4, nthread=4, num_leaves=54, random_state=1234,
               reg_alpha=0.436193, reg_lambda=0.479169, silent=-1, subsample=1,
               subsample_for_bin=240000, verbose=1)

In [11]:
y_pred_lgmb = clf_lgbm.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred_lgmb)

0.7649790136960927

In [12]:
clf_xgb = XGBClassifier(**XGBOOST_PARAMS)

In [13]:
clf_xgb.fit(X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        eval_metric= 'auc', 
        verbose=1, 
        early_stopping_rounds=200)

[10:17:23] WARNING: ../src/learner.cc:516: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.67437	validation_1-auc:0.66868
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.71135	validation_1-auc:0.70917
[2]	validation_0-auc:0.71306	validation_1-auc:0.71253
[3]	validation_0-auc:0.72232	validation_1-auc:0.72071
[4]	validation_0-auc:0.72365	validation_1-auc:0.72434
[5]	validation_0-auc:0.72514	validation_1-auc:0.72469
[6]	validation_0-auc:0.72401	validation_1-auc:0.72429
[7]	validation_0-auc:0.72303	validation_1-auc:0.72387
[8]	validation_0-auc:0.72257	validation_1-auc:0.72379
[9]	validation_0-auc:0.72269	v

[140]	validation_0-auc:0.73534	validation_1-auc:0.73371
[141]	validation_0-auc:0.73531	validation_1-auc:0.73368
[142]	validation_0-auc:0.73533	validation_1-auc:0.73368
[143]	validation_0-auc:0.73530	validation_1-auc:0.73367
[144]	validation_0-auc:0.73529	validation_1-auc:0.73367
[145]	validation_0-auc:0.73546	validation_1-auc:0.73382
[146]	validation_0-auc:0.73549	validation_1-auc:0.73384
[147]	validation_0-auc:0.73549	validation_1-auc:0.73382
[148]	validation_0-auc:0.73547	validation_1-auc:0.73375
[149]	validation_0-auc:0.73558	validation_1-auc:0.73385
[150]	validation_0-auc:0.73570	validation_1-auc:0.73397
[151]	validation_0-auc:0.73578	validation_1-auc:0.73406
[152]	validation_0-auc:0.73580	validation_1-auc:0.73406
[153]	validation_0-auc:0.73582	validation_1-auc:0.73406
[154]	validation_0-auc:0.73611	validation_1-auc:0.73430
[155]	validation_0-auc:0.73623	validation_1-auc:0.73438
[156]	validation_0-auc:0.73631	validation_1-auc:0.73443
[157]	validation_0-auc:0.73633	validation_1-auc:

[287]	validation_0-auc:0.74593	validation_1-auc:0.74214
[288]	validation_0-auc:0.74601	validation_1-auc:0.74220
[289]	validation_0-auc:0.74613	validation_1-auc:0.74231
[290]	validation_0-auc:0.74624	validation_1-auc:0.74238
[291]	validation_0-auc:0.74637	validation_1-auc:0.74251
[292]	validation_0-auc:0.74647	validation_1-auc:0.74257
[293]	validation_0-auc:0.74654	validation_1-auc:0.74261
[294]	validation_0-auc:0.74667	validation_1-auc:0.74273
[295]	validation_0-auc:0.74681	validation_1-auc:0.74286
[296]	validation_0-auc:0.74690	validation_1-auc:0.74292
[297]	validation_0-auc:0.74697	validation_1-auc:0.74296
[298]	validation_0-auc:0.74704	validation_1-auc:0.74304
[299]	validation_0-auc:0.74714	validation_1-auc:0.74313
[300]	validation_0-auc:0.74720	validation_1-auc:0.74315
[301]	validation_0-auc:0.74726	validation_1-auc:0.74318
[302]	validation_0-auc:0.74737	validation_1-auc:0.74326
[303]	validation_0-auc:0.74747	validation_1-auc:0.74335
[304]	validation_0-auc:0.74750	validation_1-auc:

[434]	validation_0-auc:0.75665	validation_1-auc:0.75047
[435]	validation_0-auc:0.75668	validation_1-auc:0.75050
[436]	validation_0-auc:0.75671	validation_1-auc:0.75050
[437]	validation_0-auc:0.75676	validation_1-auc:0.75053
[438]	validation_0-auc:0.75682	validation_1-auc:0.75057
[439]	validation_0-auc:0.75688	validation_1-auc:0.75063
[440]	validation_0-auc:0.75698	validation_1-auc:0.75072
[441]	validation_0-auc:0.75703	validation_1-auc:0.75076
[442]	validation_0-auc:0.75707	validation_1-auc:0.75078
[443]	validation_0-auc:0.75713	validation_1-auc:0.75082
[444]	validation_0-auc:0.75717	validation_1-auc:0.75086
[445]	validation_0-auc:0.75723	validation_1-auc:0.75092
[446]	validation_0-auc:0.75727	validation_1-auc:0.75095
[447]	validation_0-auc:0.75731	validation_1-auc:0.75100
[448]	validation_0-auc:0.75737	validation_1-auc:0.75107
[449]	validation_0-auc:0.75742	validation_1-auc:0.75111
[450]	validation_0-auc:0.75751	validation_1-auc:0.75116
[451]	validation_0-auc:0.75756	validation_1-auc:

[581]	validation_0-auc:0.76383	validation_1-auc:0.75533
[582]	validation_0-auc:0.76386	validation_1-auc:0.75535
[583]	validation_0-auc:0.76390	validation_1-auc:0.75537
[584]	validation_0-auc:0.76394	validation_1-auc:0.75537
[585]	validation_0-auc:0.76399	validation_1-auc:0.75541
[586]	validation_0-auc:0.76403	validation_1-auc:0.75545
[587]	validation_0-auc:0.76407	validation_1-auc:0.75547
[588]	validation_0-auc:0.76410	validation_1-auc:0.75548
[589]	validation_0-auc:0.76414	validation_1-auc:0.75549
[590]	validation_0-auc:0.76419	validation_1-auc:0.75554
[591]	validation_0-auc:0.76422	validation_1-auc:0.75556
[592]	validation_0-auc:0.76424	validation_1-auc:0.75558
[593]	validation_0-auc:0.76428	validation_1-auc:0.75560
[594]	validation_0-auc:0.76432	validation_1-auc:0.75564
[595]	validation_0-auc:0.76437	validation_1-auc:0.75567
[596]	validation_0-auc:0.76440	validation_1-auc:0.75570
[597]	validation_0-auc:0.76444	validation_1-auc:0.75574
[598]	validation_0-auc:0.76449	validation_1-auc:

...

[1453]	validation_0-auc:0.78416	validation_1-auc:0.76509
[1454]	validation_0-auc:0.78417	validation_1-auc:0.76509
[1455]	validation_0-auc:0.78418	validation_1-auc:0.76511
[1456]	validation_0-auc:0.78421	validation_1-auc:0.76512
[1457]	validation_0-auc:0.78423	validation_1-auc:0.76512
[1458]	validation_0-auc:0.78425	validation_1-auc:0.76513
[1459]	validation_0-auc:0.78427	validation_1-auc:0.76514
[1460]	validation_0-auc:0.78430	validation_1-auc:0.76514
[1461]	validation_0-auc:0.78431	validation_1-auc:0.76514
[1462]	validation_0-auc:0.78432	validation_1-auc:0.76514
[1463]	validation_0-auc:0.78434	validation_1-auc:0.76514
[1464]	validation_0-auc:0.78434	validation_1-auc:0.76514
[1465]	validation_0-auc:0.78437	validation_1-auc:0.76514
[1466]	validation_0-auc:0.78438	validation_1-auc:0.76514
[1467]	validation_0-auc:0.78438	validation_1-auc:0.76513
[1468]	validation_0-auc:0.78440	validation_1-auc:0.76513
[1469]	validation_0-auc:0.78442	validation_1-auc:0.76513
[1470]	validation_0-auc:0.78444

[1597]	validation_0-auc:0.78634	validation_1-auc:0.76578
[1598]	validation_0-auc:0.78636	validation_1-auc:0.76578
[1599]	validation_0-auc:0.78636	validation_1-auc:0.76578
[1600]	validation_0-auc:0.78639	validation_1-auc:0.76579
[1601]	validation_0-auc:0.78641	validation_1-auc:0.76578
[1602]	validation_0-auc:0.78642	validation_1-auc:0.76578
[1603]	validation_0-auc:0.78643	validation_1-auc:0.76579
[1604]	validation_0-auc:0.78645	validation_1-auc:0.76580
[1605]	validation_0-auc:0.78648	validation_1-auc:0.76583
[1606]	validation_0-auc:0.78651	validation_1-auc:0.76584
[1607]	validation_0-auc:0.78653	validation_1-auc:0.76584
[1608]	validation_0-auc:0.78655	validation_1-auc:0.76586
[1609]	validation_0-auc:0.78658	validation_1-auc:0.76587
[1610]	validation_0-auc:0.78660	validation_1-auc:0.76587
[1611]	validation_0-auc:0.78661	validation_1-auc:0.76588
[1612]	validation_0-auc:0.78662	validation_1-auc:0.76588
[1613]	validation_0-auc:0.78664	validation_1-auc:0.76589
[1614]	validation_0-auc:0.78665

[1741]	validation_0-auc:0.78853	validation_1-auc:0.76634
[1742]	validation_0-auc:0.78854	validation_1-auc:0.76634
[1743]	validation_0-auc:0.78855	validation_1-auc:0.76634
[1744]	validation_0-auc:0.78857	validation_1-auc:0.76634
[1745]	validation_0-auc:0.78859	validation_1-auc:0.76635
[1746]	validation_0-auc:0.78860	validation_1-auc:0.76635
[1747]	validation_0-auc:0.78861	validation_1-auc:0.76635
[1748]	validation_0-auc:0.78863	validation_1-auc:0.76634
[1749]	validation_0-auc:0.78864	validation_1-auc:0.76634
[1750]	validation_0-auc:0.78866	validation_1-auc:0.76635
[1751]	validation_0-auc:0.78866	validation_1-auc:0.76636
[1752]	validation_0-auc:0.78868	validation_1-auc:0.76636
[1753]	validation_0-auc:0.78869	validation_1-auc:0.76635
[1754]	validation_0-auc:0.78870	validation_1-auc:0.76635
[1755]	validation_0-auc:0.78872	validation_1-auc:0.76635
[1756]	validation_0-auc:0.78873	validation_1-auc:0.76635
[1757]	validation_0-auc:0.78874	validation_1-auc:0.76636
[1758]	validation_0-auc:0.78876

[1885]	validation_0-auc:0.79051	validation_1-auc:0.76674
[1886]	validation_0-auc:0.79052	validation_1-auc:0.76675
[1887]	validation_0-auc:0.79053	validation_1-auc:0.76675
[1888]	validation_0-auc:0.79054	validation_1-auc:0.76674
[1889]	validation_0-auc:0.79054	validation_1-auc:0.76675
[1890]	validation_0-auc:0.79056	validation_1-auc:0.76675
[1891]	validation_0-auc:0.79057	validation_1-auc:0.76675
[1892]	validation_0-auc:0.79057	validation_1-auc:0.76675
[1893]	validation_0-auc:0.79059	validation_1-auc:0.76675
[1894]	validation_0-auc:0.79059	validation_1-auc:0.76675
[1895]	validation_0-auc:0.79060	validation_1-auc:0.76675
[1896]	validation_0-auc:0.79061	validation_1-auc:0.76675
[1897]	validation_0-auc:0.79063	validation_1-auc:0.76676
[1898]	validation_0-auc:0.79065	validation_1-auc:0.76677
[1899]	validation_0-auc:0.79066	validation_1-auc:0.76677
[1900]	validation_0-auc:0.79067	validation_1-auc:0.76677
[1901]	validation_0-auc:0.79069	validation_1-auc:0.76678
[1902]	validation_0-auc:0.79070

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=4, metric='auc',
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=0, num_parallel_tree=1,
              random_state=1234, reg_alpha=0, reg_lambda=1.2,
              scale_pos_weight=2.5, seed=1234, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
y_pred_xgb = clf_xgb.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred_xgb)

0.7643793080006586

In [15]:
clf_tabnet = TabNetClassifier(optimizer_fn=torch.optim.Adam)

Device used : cpu


/home/aasleptsov98/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [16]:
max_epochs = 25 if not os.getenv("CI", False) else 2

In [17]:
clf_tabnet.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
) 

epoch 0  | loss: 0.6905  | train_auc: 0.66384 | valid_auc: 0.66344 |  0:01:48s
epoch 1  | loss: 0.62888 | train_auc: 0.72893 | valid_auc: 0.72733 |  0:03:38s
epoch 2  | loss: 0.60947 | train_auc: 0.73841 | valid_auc: 0.73566 |  0:05:28s
epoch 3  | loss: 0.60188 | train_auc: 0.74774 | valid_auc: 0.74183 |  0:07:17s
epoch 4  | loss: 0.5956  | train_auc: 0.75284 | valid_auc: 0.74171 |  0:09:04s
epoch 5  | loss: 0.59021 | train_auc: 0.75728 | valid_auc: 0.74259 |  0:10:52s
epoch 6  | loss: 0.58458 | train_auc: 0.7648  | valid_auc: 0.74348 |  0:12:39s
epoch 7  | loss: 0.58018 | train_auc: 0.77018 | valid_auc: 0.74079 |  0:14:25s
epoch 8  | loss: 0.57314 | train_auc: 0.7745  | valid_auc: 0.74698 |  0:16:12s
epoch 9  | loss: 0.56927 | train_auc: 0.77999 | valid_auc: 0.74271 |  0:18:01s
epoch 10 | loss: 0.56218 | train_auc: 0.78514 | valid_auc: 0.73634 |  0:19:48s
epoch 11 | loss: 0.56022 | train_auc: 0.78746 | valid_auc: 0.73395 |  0:21:35s
epoch 12 | loss: 0.55519 | train_auc: 0.78954 | vali

KeyboardInterrupt: 

In [18]:
y_pred_tabnet = clf_tabnet.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred_tabnet)

0.728675276184152

# All data

In [13]:
num_rows = None
df = application_train_test(num_rows)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')
    del bureau
    gc.collect()
with timer("Process previous_applications"):
    prev = previous_applications(num_rows)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')
    del prev
    gc.collect()
with timer("Process POS-CASH balance"):
    pos = pos_cash(num_rows)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')
    del pos
    gc.collect()
with timer("Process installments payments"):
    ins = installments_payments(num_rows)
    print("Installments payments df shape:", ins.shape)
    df = df.join(ins, how='left', on='SK_ID_CURR')
    del ins
    gc.collect()
with timer("Process credit card balance"):
    cc = credit_card_balance(num_rows)
    print("Credit card balance df shape:", cc.shape)
    df = df.join(cc, how='left', on='SK_ID_CURR')
    del cc
    gc.collect()

Train samples: 307511
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 29s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 13s
Installments payments df shape: (339587, 26)
Process installments payments - done in 41s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 19s


In [14]:
df= df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

In [15]:
X = df.drop(['TARGET','SK_ID_CURR'], axis=1).values
y = df["TARGET"].values

In [16]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [17]:
clf_lgbm = LGBMClassifier(**LIGHTGBM_PARAMS)

In [18]:
clf_lgbm.fit(X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        eval_metric= 'auc', 
        verbose=1, 
        early_stopping_rounds=200)

[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=4. Current value: num_threads=4
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=4. Current value: num_threads=4
[LightGBM] [Info] Number of positive: 19804, number of negative: 226204
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.287029 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 64553
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 491
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=4. Current value: num_threads=4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080501 -> initscore=-2.435553
[LightGBM] [Info] Start training from score -2.435553
[1]	training's auc: 0.713635	valid_1's auc: 0.705196
Tr

[136]	training's auc: 0.770032	valid_1's auc: 0.753341
[137]	training's auc: 0.77011	valid_1's auc: 0.753341
[138]	training's auc: 0.770206	valid_1's auc: 0.753427
[139]	training's auc: 0.77028	valid_1's auc: 0.75346
[140]	training's auc: 0.770329	valid_1's auc: 0.753472
[141]	training's auc: 0.770316	valid_1's auc: 0.753452
[142]	training's auc: 0.770363	valid_1's auc: 0.753517
[143]	training's auc: 0.770455	valid_1's auc: 0.753588
[144]	training's auc: 0.770542	valid_1's auc: 0.753673
[145]	training's auc: 0.770595	valid_1's auc: 0.753653
[146]	training's auc: 0.770662	valid_1's auc: 0.753735
[147]	training's auc: 0.770674	valid_1's auc: 0.753756
[148]	training's auc: 0.77074	valid_1's auc: 0.753772
[149]	training's auc: 0.770835	valid_1's auc: 0.753812
[150]	training's auc: 0.77083	valid_1's auc: 0.753815
[151]	training's auc: 0.770909	valid_1's auc: 0.753955
[152]	training's auc: 0.771021	valid_1's auc: 0.754009
[153]	training's auc: 0.77109	valid_1's auc: 0.754118
[154]	training's

[286]	training's auc: 0.780252	valid_1's auc: 0.758575
[287]	training's auc: 0.780326	valid_1's auc: 0.758571
[288]	training's auc: 0.780398	valid_1's auc: 0.758628
[289]	training's auc: 0.780492	valid_1's auc: 0.758643
[290]	training's auc: 0.78056	valid_1's auc: 0.758659
[291]	training's auc: 0.780583	valid_1's auc: 0.758686
[292]	training's auc: 0.780641	valid_1's auc: 0.758722
[293]	training's auc: 0.780696	valid_1's auc: 0.758721
[294]	training's auc: 0.780747	valid_1's auc: 0.758775
[295]	training's auc: 0.780861	valid_1's auc: 0.75886
[296]	training's auc: 0.78099	valid_1's auc: 0.75891
[297]	training's auc: 0.78103	valid_1's auc: 0.758959
[298]	training's auc: 0.781088	valid_1's auc: 0.758976
[299]	training's auc: 0.781151	valid_1's auc: 0.759007
[300]	training's auc: 0.781214	valid_1's auc: 0.759026
[301]	training's auc: 0.781276	valid_1's auc: 0.759079
[302]	training's auc: 0.781409	valid_1's auc: 0.759166
[303]	training's auc: 0.781473	valid_1's auc: 0.759181
[304]	training'

[436]	training's auc: 0.78977	valid_1's auc: 0.762559
[437]	training's auc: 0.789838	valid_1's auc: 0.762595
[438]	training's auc: 0.789907	valid_1's auc: 0.762622
[439]	training's auc: 0.789932	valid_1's auc: 0.762618
[440]	training's auc: 0.789973	valid_1's auc: 0.76262
[441]	training's auc: 0.790024	valid_1's auc: 0.762633
[442]	training's auc: 0.790102	valid_1's auc: 0.762673
[443]	training's auc: 0.790134	valid_1's auc: 0.762687
[444]	training's auc: 0.790209	valid_1's auc: 0.762734
[445]	training's auc: 0.790264	valid_1's auc: 0.762775
[446]	training's auc: 0.790302	valid_1's auc: 0.762796
[447]	training's auc: 0.790389	valid_1's auc: 0.762851
[448]	training's auc: 0.790461	valid_1's auc: 0.762878
[449]	training's auc: 0.790533	valid_1's auc: 0.762904
[450]	training's auc: 0.790576	valid_1's auc: 0.762939
[451]	training's auc: 0.790648	valid_1's auc: 0.762982
[452]	training's auc: 0.790714	valid_1's auc: 0.763025
[453]	training's auc: 0.790757	valid_1's auc: 0.763053
[454]	traini

[586]	training's auc: 0.798367	valid_1's auc: 0.765962
[587]	training's auc: 0.798419	valid_1's auc: 0.765985
[588]	training's auc: 0.798471	valid_1's auc: 0.766012
[589]	training's auc: 0.798522	valid_1's auc: 0.766059
[590]	training's auc: 0.798596	valid_1's auc: 0.766077
[591]	training's auc: 0.798653	valid_1's auc: 0.766092
[592]	training's auc: 0.798712	valid_1's auc: 0.766107
[593]	training's auc: 0.798754	valid_1's auc: 0.766116
[594]	training's auc: 0.798803	valid_1's auc: 0.76615
[595]	training's auc: 0.798869	valid_1's auc: 0.766151
[596]	training's auc: 0.798911	valid_1's auc: 0.766158
[597]	training's auc: 0.79895	valid_1's auc: 0.766171
[598]	training's auc: 0.798989	valid_1's auc: 0.766182
[599]	training's auc: 0.799022	valid_1's auc: 0.766186
[600]	training's auc: 0.799068	valid_1's auc: 0.76622
[601]	training's auc: 0.799113	valid_1's auc: 0.766247
[602]	training's auc: 0.799163	valid_1's auc: 0.766261
[603]	training's auc: 0.79922	valid_1's auc: 0.76626
[604]	training'

...

[1477]	training's auc: 0.838004	valid_1's auc: 0.77627
[1478]	training's auc: 0.838046	valid_1's auc: 0.776265
[1479]	training's auc: 0.838082	valid_1's auc: 0.776274
[1480]	training's auc: 0.838124	valid_1's auc: 0.776283
[1481]	training's auc: 0.838145	valid_1's auc: 0.776291
[1482]	training's auc: 0.838191	valid_1's auc: 0.776303
[1483]	training's auc: 0.838226	valid_1's auc: 0.776317
[1484]	training's auc: 0.838258	valid_1's auc: 0.776325
[1485]	training's auc: 0.838287	valid_1's auc: 0.776339
[1486]	training's auc: 0.83833	valid_1's auc: 0.776336
[1487]	training's auc: 0.838369	valid_1's auc: 0.776342
[1488]	training's auc: 0.838414	valid_1's auc: 0.776346
[1489]	training's auc: 0.838437	valid_1's auc: 0.77635
[1490]	training's auc: 0.838471	valid_1's auc: 0.776346
[1491]	training's auc: 0.838502	valid_1's auc: 0.776348
[1492]	training's auc: 0.838533	valid_1's auc: 0.776356
[1493]	training's auc: 0.83858	valid_1's auc: 0.77637
[1494]	training's auc: 0.838621	valid_1's auc: 0.7763

[1624]	training's auc: 0.843115	valid_1's auc: 0.777182
[1625]	training's auc: 0.843156	valid_1's auc: 0.77719
[1626]	training's auc: 0.843185	valid_1's auc: 0.777202
[1627]	training's auc: 0.843217	valid_1's auc: 0.777207
[1628]	training's auc: 0.843252	valid_1's auc: 0.777225
[1629]	training's auc: 0.843285	valid_1's auc: 0.777242
[1630]	training's auc: 0.843312	valid_1's auc: 0.777247
[1631]	training's auc: 0.843345	valid_1's auc: 0.777251
[1632]	training's auc: 0.843389	valid_1's auc: 0.777254
[1633]	training's auc: 0.843418	valid_1's auc: 0.777275
[1634]	training's auc: 0.843451	valid_1's auc: 0.777282
[1635]	training's auc: 0.843482	valid_1's auc: 0.777283
[1636]	training's auc: 0.84352	valid_1's auc: 0.777294
[1637]	training's auc: 0.843552	valid_1's auc: 0.777299
[1638]	training's auc: 0.843584	valid_1's auc: 0.777298
[1639]	training's auc: 0.843628	valid_1's auc: 0.777288
[1640]	training's auc: 0.843666	valid_1's auc: 0.777292
[1641]	training's auc: 0.843693	valid_1's auc: 0.7

[1771]	training's auc: 0.848055	valid_1's auc: 0.777984
[1772]	training's auc: 0.848082	valid_1's auc: 0.777988
[1773]	training's auc: 0.848112	valid_1's auc: 0.777985
[1774]	training's auc: 0.848145	valid_1's auc: 0.777987
[1775]	training's auc: 0.84817	valid_1's auc: 0.777991
[1776]	training's auc: 0.848202	valid_1's auc: 0.777993
[1777]	training's auc: 0.848244	valid_1's auc: 0.778008
[1778]	training's auc: 0.848282	valid_1's auc: 0.77801
[1779]	training's auc: 0.848318	valid_1's auc: 0.778004
[1780]	training's auc: 0.848359	valid_1's auc: 0.778
[1781]	training's auc: 0.848403	valid_1's auc: 0.778009
[1782]	training's auc: 0.848436	valid_1's auc: 0.77802
[1783]	training's auc: 0.848469	valid_1's auc: 0.778032
[1784]	training's auc: 0.848502	valid_1's auc: 0.778039
[1785]	training's auc: 0.848543	valid_1's auc: 0.778046
[1786]	training's auc: 0.848581	valid_1's auc: 0.778051
[1787]	training's auc: 0.848614	valid_1's auc: 0.778057
[1788]	training's auc: 0.848643	valid_1's auc: 0.77807

[1919]	training's auc: 0.852712	valid_1's auc: 0.778709
[1920]	training's auc: 0.852744	valid_1's auc: 0.77871
[1921]	training's auc: 0.852793	valid_1's auc: 0.778722
[1922]	training's auc: 0.852824	valid_1's auc: 0.778728
[1923]	training's auc: 0.852858	valid_1's auc: 0.778724
[1924]	training's auc: 0.852888	valid_1's auc: 0.778733
[1925]	training's auc: 0.852914	valid_1's auc: 0.778739
[1926]	training's auc: 0.852938	valid_1's auc: 0.778736
[1927]	training's auc: 0.852967	valid_1's auc: 0.778732
[1928]	training's auc: 0.853001	valid_1's auc: 0.778737
[1929]	training's auc: 0.853034	valid_1's auc: 0.778744
[1930]	training's auc: 0.853072	valid_1's auc: 0.778742
[1931]	training's auc: 0.853101	valid_1's auc: 0.778759
[1932]	training's auc: 0.853134	valid_1's auc: 0.778768
[1933]	training's auc: 0.853171	valid_1's auc: 0.778767
[1934]	training's auc: 0.853196	valid_1's auc: 0.77876
[1935]	training's auc: 0.853237	valid_1's auc: 0.778764
[1936]	training's auc: 0.85327	valid_1's auc: 0.77

LGBMClassifier(boosting_type='goss', colsample_bytree=0.508716,
               is_unbalance=False, learning_rate=0.005134, max_depth=10,
               metric='auc', min_split_gain=0.024766, n_estimators=2000,
               n_jobs=4, nthread=4, num_leaves=54, random_state=1234,
               reg_alpha=0.436193, reg_lambda=0.479169, silent=-1, subsample=1,
               subsample_for_bin=240000, verbose=1)

In [64]:
y_pred_lgmb = clf_lgbm.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred_lgmb)

0.7833630731456633

In [20]:
clf_xgb = XGBClassifier(**XGBOOST_PARAMS)

In [21]:
clf_xgb.fit(X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        eval_metric= 'auc', 
        verbose=1, 
        early_stopping_rounds=200)

[05:02:07] WARNING: ../src/learner.cc:516: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.69818	validation_1-auc:0.69786
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.70315	validation_1-auc:0.70291
[2]	validation_0-auc:0.70886	validation_1-auc:0.70834
[3]	validation_0-auc:0.70947	validation_1-auc:0.70894
[4]	validation_0-auc:0.71910	validation_1-auc:0.71649
[5]	validation_0-auc:0.71832	validation_1-auc:0.71696
[6]	validation_0-auc:0.71925	validation_1-auc:0.71701
[7]	validation_0-auc:0.71898	validation_1-auc:0.71759
[8]	validation_0-auc:0.71812	validation_1-auc:0.71695
[9]	validation_0-auc:0.71794	v

[140]	validation_0-auc:0.73959	validation_1-auc:0.73388
[141]	validation_0-auc:0.73970	validation_1-auc:0.73401
[142]	validation_0-auc:0.73972	validation_1-auc:0.73401
[143]	validation_0-auc:0.73980	validation_1-auc:0.73399
[144]	validation_0-auc:0.73995	validation_1-auc:0.73409
[145]	validation_0-auc:0.73995	validation_1-auc:0.73411
[146]	validation_0-auc:0.74000	validation_1-auc:0.73411
[147]	validation_0-auc:0.73998	validation_1-auc:0.73410
[148]	validation_0-auc:0.74004	validation_1-auc:0.73418
[149]	validation_0-auc:0.74018	validation_1-auc:0.73430
[150]	validation_0-auc:0.74033	validation_1-auc:0.73438
[151]	validation_0-auc:0.74041	validation_1-auc:0.73445
[152]	validation_0-auc:0.74042	validation_1-auc:0.73442
[153]	validation_0-auc:0.74046	validation_1-auc:0.73444
[154]	validation_0-auc:0.74061	validation_1-auc:0.73457
[155]	validation_0-auc:0.74062	validation_1-auc:0.73454
[156]	validation_0-auc:0.74076	validation_1-auc:0.73467
[157]	validation_0-auc:0.74079	validation_1-auc:

[287]	validation_0-auc:0.75352	validation_1-auc:0.74399
[288]	validation_0-auc:0.75363	validation_1-auc:0.74406
[289]	validation_0-auc:0.75376	validation_1-auc:0.74419
[290]	validation_0-auc:0.75383	validation_1-auc:0.74423
[291]	validation_0-auc:0.75392	validation_1-auc:0.74428
[292]	validation_0-auc:0.75401	validation_1-auc:0.74436
[293]	validation_0-auc:0.75419	validation_1-auc:0.74450
[294]	validation_0-auc:0.75430	validation_1-auc:0.74459
[295]	validation_0-auc:0.75446	validation_1-auc:0.74476
[296]	validation_0-auc:0.75461	validation_1-auc:0.74484
[297]	validation_0-auc:0.75473	validation_1-auc:0.74492
[298]	validation_0-auc:0.75485	validation_1-auc:0.74501
[299]	validation_0-auc:0.75500	validation_1-auc:0.74510
[300]	validation_0-auc:0.75514	validation_1-auc:0.74523
[301]	validation_0-auc:0.75526	validation_1-auc:0.74533
[302]	validation_0-auc:0.75538	validation_1-auc:0.74542
[303]	validation_0-auc:0.75549	validation_1-auc:0.74551
[304]	validation_0-auc:0.75559	validation_1-auc:

[434]	validation_0-auc:0.76738	validation_1-auc:0.75414
[435]	validation_0-auc:0.76746	validation_1-auc:0.75421
[436]	validation_0-auc:0.76754	validation_1-auc:0.75426
[437]	validation_0-auc:0.76762	validation_1-auc:0.75431
[438]	validation_0-auc:0.76772	validation_1-auc:0.75440
[439]	validation_0-auc:0.76781	validation_1-auc:0.75448
[440]	validation_0-auc:0.76787	validation_1-auc:0.75450
[441]	validation_0-auc:0.76792	validation_1-auc:0.75452
[442]	validation_0-auc:0.76802	validation_1-auc:0.75462
[443]	validation_0-auc:0.76808	validation_1-auc:0.75465
[444]	validation_0-auc:0.76816	validation_1-auc:0.75469
[445]	validation_0-auc:0.76822	validation_1-auc:0.75475
[446]	validation_0-auc:0.76828	validation_1-auc:0.75479
[447]	validation_0-auc:0.76835	validation_1-auc:0.75484
[448]	validation_0-auc:0.76844	validation_1-auc:0.75489
[449]	validation_0-auc:0.76851	validation_1-auc:0.75491
[450]	validation_0-auc:0.76857	validation_1-auc:0.75495
[451]	validation_0-auc:0.76866	validation_1-auc:

[581]	validation_0-auc:0.77615	validation_1-auc:0.76039
[582]	validation_0-auc:0.77620	validation_1-auc:0.76043
[583]	validation_0-auc:0.77625	validation_1-auc:0.76045
[584]	validation_0-auc:0.77633	validation_1-auc:0.76052
[585]	validation_0-auc:0.77639	validation_1-auc:0.76055
[586]	validation_0-auc:0.77644	validation_1-auc:0.76060
[587]	validation_0-auc:0.77647	validation_1-auc:0.76061
[588]	validation_0-auc:0.77653	validation_1-auc:0.76064
[589]	validation_0-auc:0.77659	validation_1-auc:0.76065
[590]	validation_0-auc:0.77662	validation_1-auc:0.76068
[591]	validation_0-auc:0.77668	validation_1-auc:0.76070
[592]	validation_0-auc:0.77672	validation_1-auc:0.76073
[593]	validation_0-auc:0.77675	validation_1-auc:0.76077
[594]	validation_0-auc:0.77680	validation_1-auc:0.76080
[595]	validation_0-auc:0.77683	validation_1-auc:0.76082
[596]	validation_0-auc:0.77688	validation_1-auc:0.76087
[597]	validation_0-auc:0.77693	validation_1-auc:0.76090
[598]	validation_0-auc:0.77698	validation_1-auc:

...

[1453]	validation_0-auc:0.80167	validation_1-auc:0.77349
[1454]	validation_0-auc:0.80169	validation_1-auc:0.77352
[1455]	validation_0-auc:0.80171	validation_1-auc:0.77352
[1456]	validation_0-auc:0.80173	validation_1-auc:0.77353
[1457]	validation_0-auc:0.80175	validation_1-auc:0.77352
[1458]	validation_0-auc:0.80177	validation_1-auc:0.77353
[1459]	validation_0-auc:0.80178	validation_1-auc:0.77355
[1460]	validation_0-auc:0.80180	validation_1-auc:0.77356
[1461]	validation_0-auc:0.80182	validation_1-auc:0.77357
[1462]	validation_0-auc:0.80184	validation_1-auc:0.77357
[1463]	validation_0-auc:0.80186	validation_1-auc:0.77359
[1464]	validation_0-auc:0.80188	validation_1-auc:0.77359
[1465]	validation_0-auc:0.80190	validation_1-auc:0.77359
[1466]	validation_0-auc:0.80192	validation_1-auc:0.77360
[1467]	validation_0-auc:0.80194	validation_1-auc:0.77362
[1468]	validation_0-auc:0.80196	validation_1-auc:0.77361
[1469]	validation_0-auc:0.80198	validation_1-auc:0.77360
[1470]	validation_0-auc:0.80200

[1597]	validation_0-auc:0.80459	validation_1-auc:0.77465
[1598]	validation_0-auc:0.80461	validation_1-auc:0.77466
[1599]	validation_0-auc:0.80463	validation_1-auc:0.77467
[1600]	validation_0-auc:0.80465	validation_1-auc:0.77467
[1601]	validation_0-auc:0.80467	validation_1-auc:0.77467
[1602]	validation_0-auc:0.80470	validation_1-auc:0.77467
[1603]	validation_0-auc:0.80471	validation_1-auc:0.77468
[1604]	validation_0-auc:0.80473	validation_1-auc:0.77469
[1605]	validation_0-auc:0.80475	validation_1-auc:0.77470
[1606]	validation_0-auc:0.80478	validation_1-auc:0.77471
[1607]	validation_0-auc:0.80480	validation_1-auc:0.77472
[1608]	validation_0-auc:0.80481	validation_1-auc:0.77472
[1609]	validation_0-auc:0.80482	validation_1-auc:0.77473
[1610]	validation_0-auc:0.80484	validation_1-auc:0.77474
[1611]	validation_0-auc:0.80486	validation_1-auc:0.77473
[1612]	validation_0-auc:0.80488	validation_1-auc:0.77475
[1613]	validation_0-auc:0.80492	validation_1-auc:0.77477
[1614]	validation_0-auc:0.80494

[1741]	validation_0-auc:0.80726	validation_1-auc:0.77549
[1742]	validation_0-auc:0.80728	validation_1-auc:0.77550
[1743]	validation_0-auc:0.80729	validation_1-auc:0.77550
[1744]	validation_0-auc:0.80732	validation_1-auc:0.77551
[1745]	validation_0-auc:0.80734	validation_1-auc:0.77554
[1746]	validation_0-auc:0.80736	validation_1-auc:0.77555
[1747]	validation_0-auc:0.80737	validation_1-auc:0.77555
[1748]	validation_0-auc:0.80739	validation_1-auc:0.77555
[1749]	validation_0-auc:0.80740	validation_1-auc:0.77556
[1750]	validation_0-auc:0.80742	validation_1-auc:0.77557
[1751]	validation_0-auc:0.80744	validation_1-auc:0.77557
[1752]	validation_0-auc:0.80745	validation_1-auc:0.77557
[1753]	validation_0-auc:0.80747	validation_1-auc:0.77557
[1754]	validation_0-auc:0.80749	validation_1-auc:0.77557
[1755]	validation_0-auc:0.80750	validation_1-auc:0.77557
[1756]	validation_0-auc:0.80752	validation_1-auc:0.77559
[1757]	validation_0-auc:0.80754	validation_1-auc:0.77558
[1758]	validation_0-auc:0.80756

[1885]	validation_0-auc:0.80986	validation_1-auc:0.77628
[1886]	validation_0-auc:0.80988	validation_1-auc:0.77628
[1887]	validation_0-auc:0.80989	validation_1-auc:0.77629
[1888]	validation_0-auc:0.80991	validation_1-auc:0.77628
[1889]	validation_0-auc:0.80993	validation_1-auc:0.77630
[1890]	validation_0-auc:0.80995	validation_1-auc:0.77632
[1891]	validation_0-auc:0.80997	validation_1-auc:0.77632
[1892]	validation_0-auc:0.80999	validation_1-auc:0.77632
[1893]	validation_0-auc:0.81001	validation_1-auc:0.77632
[1894]	validation_0-auc:0.81004	validation_1-auc:0.77633
[1895]	validation_0-auc:0.81005	validation_1-auc:0.77633
[1896]	validation_0-auc:0.81007	validation_1-auc:0.77634
[1897]	validation_0-auc:0.81008	validation_1-auc:0.77634
[1898]	validation_0-auc:0.81010	validation_1-auc:0.77634
[1899]	validation_0-auc:0.81012	validation_1-auc:0.77634
[1900]	validation_0-auc:0.81014	validation_1-auc:0.77634
[1901]	validation_0-auc:0.81015	validation_1-auc:0.77635
[1902]	validation_0-auc:0.81017

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=4, metric='auc',
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=0, num_parallel_tree=1,
              random_state=1234, reg_alpha=0, reg_lambda=1.2,
              scale_pos_weight=2.5, seed=1234, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
y_pred_xgb = clf_xgb.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred_xgb)

0.7817701318236424

In [23]:
clf_tabnet = TabNetClassifier(optimizer_fn=torch.optim.Adam)

Device used : cpu


/home/aasleptsov98/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [24]:
max_epochs = 25 if not os.getenv("CI", False) else 2

In [25]:
clf_tabnet.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
) 

epoch 0  | loss: 0.67073 | train_auc: 0.64046 | valid_auc: 0.63228 |  0:02:28s
epoch 1  | loss: 0.62993 | train_auc: 0.71977 | valid_auc: 0.71356 |  0:04:56s
epoch 2  | loss: 0.60818 | train_auc: 0.74126 | valid_auc: 0.73549 |  0:07:23s
epoch 3  | loss: 0.60457 | train_auc: 0.74879 | valid_auc: 0.74114 |  0:09:52s
epoch 4  | loss: 0.59455 | train_auc: 0.73424 | valid_auc: 0.72791 |  0:12:18s
epoch 5  | loss: 0.60334 | train_auc: 0.73765 | valid_auc: 0.72843 |  0:14:43s
epoch 6  | loss: 0.60719 | train_auc: 0.73722 | valid_auc: 0.73084 |  0:17:03s
epoch 7  | loss: 0.5998  | train_auc: 0.72974 | valid_auc: 0.72599 |  0:19:26s
epoch 8  | loss: 0.60899 | train_auc: 0.7431  | valid_auc: 0.73862 |  0:21:47s
epoch 9  | loss: 0.60028 | train_auc: 0.7385  | valid_auc: 0.72843 |  0:24:11s
epoch 10 | loss: 0.59076 | train_auc: 0.75445 | valid_auc: 0.74538 |  0:26:34s
epoch 11 | loss: 0.59353 | train_auc: 0.75428 | valid_auc: 0.75071 |  0:29:01s
epoch 12 | loss: 0.58827 | train_auc: 0.73374 | vali

In [66]:
y_pred_tabnet = clf_tabnet.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred_tabnet)

0.7555494629759629